In [1]:
# [TODO] 
# - 游戏结束判定     (atk icon的值持续不变时)  游戏结束进入菜单判断：开始时对菜单进行采样。
# - 使用感知哈希智能出牌
# - 每天首次魔力棱镜判定
# - 体力不足判定    （感知哈希 or ai识别）   ...好像也可以进入战斗的时候等系统提示再嗑果

import win32api, win32con, win32gui, win32ui
import PIL
import time
from config import *
import numpy as np
from utils import compare_img, pic_shot, compare_RGB, echo_info, compare_img_new
LOADING_ATK_DIFF = 33

In [2]:
class Cursor(object):
    def __init__(self, init_pos):
        # init_pos：should be a tuple, set `False` to skip initing position.
        if init_pos!=False and len(init_pos)==2:
            self.move_to(init_pos)
    
    def move_to(self, pos):
        win32api.SetCursorPos(pos)
    
    def get_pos(self):
        return win32api.GetCursorPos()
    
    def click(self):
        win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, 0, 0, 0, 0)
        win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, 0, 0, 0, 0)
    
    def right_click(self):
        win32api.mouse_event(win32con.MOUSEEVENTF_RIGHTDOWN, 0, 0, 0, 0) 
        win32api.mouse_event(win32con.MOUSEEVENTF_RIGHTUP, 0, 0, 0, 0)

In [6]:
class Fgo(object):
    def __init__(self, full_screen=True, sleep=True):
        # [init by yourself] put cursor at the down-right position of the game window.
        
        if full_screen:
            self.height = win32api.GetSystemMetrics(win32con.SM_CYSCREEN)
            self.width = win32api.GetSystemMetrics(win32con.SM_CXSCREEN)
            self.scr_pos1 = (0, 0)
            self.scr_pos2 = (self.width, self.height)
            self.c = Cursor(init_pos=False)
            for x in range(5):
                echo_info('INFO', 
                          'Program will start in %d s, Please enter FULL SCREEN MODE.' % (5-x))
                time.sleep(1)
            
        else:
            self.c = Cursor(init_pos=False)
            while 1:
                in1 = input('[INFO {}] Move cursor to the top-left cornor of Fgo screen, and press ENTER(enter q to exit):'.format(
                    time.strftime('%H:%M:%S')))
                if in1 == 'q':
                    exit()
                self.scr_pos1 = self.c.get_pos()
                echo_info('INFO', 
                          'Get cursor at {}'.format(self.scr_pos1))
                
                in2 = input('[INFO {}] Move cursor to the down-right cornor of Fgo screen, and press ENTER(enter q to exit):'.format(
                    time.strftime('%H:%M:%S')))
                if in2 == 'q':
                    exit()
                self.scr_pos2 = self.c.get_pos()
                echo_info('INFO', 
                          'Get cursor at {}'.format(self.scr_pos2))
                
                res = input('[INFO {}] Continue?[y/n](y to continue, n to reset cursor):'.format(
                    time.strftime('%H:%M:%S')))
                if res == 'n':
                    continue
                else: 
                    break
            if sleep:
                for x in range(3):
                    echo_info('INFO', 
                          'Program will start in %d s, make sure the game window not covered.' % (5-x))
                    time.sleep(1)

            self.width = abs(self.scr_pos2[0] - self.scr_pos1[0])
            self.height = abs(self.scr_pos2[1] - self.scr_pos1[1])
        
        
        #---------------------sampled pix info-----------------------
        self.loading_img = PIL.Image.open('./data/loading.jpg')
        self.atk_img = PIL.Image.open('./data/atk_ico.jpg')
        
        
        
        print('\n---------------------[init over]---------------------------')
        #print('[DEBUG {}] Window width(x) = {}, height(y) = {}'.format(
        #    time.strftime('%H:%M:%S'), self.width, self.height))
        
    
    def _set(self, float_x, float_y):
        # input type: float
        # reurn the real position on the screen.
        return int(self.scr_pos1[0]+self.width*float_x), int(self.scr_pos1[1]+self.height*float_y)
    
    
    def click_act(self, float_x, float_y, sleep_time, click=True):
        pos = self._set(float_x, float_y)
        self.c.move_to(pos)
        if click:
            self.c.click()
        if not DEBUG:
            echo_info('INFO', 
                     'simulate click at position: {}'.format(pos))
        time.sleep(sleep_time)
        
    
    def enter_battle(self, supNo=8):
        # [init by yourself] put the tag of battle at the top of screen.
        
        # postion of the center of battle tag.
        bat_tag_y = 0.2740
        bat_tag_x = 0.7252
        self.click_act(bat_tag_x, bat_tag_y, 1)
        
        # choose support:
        # postion of support servant tag.
        sup_tag_x = 0.4893
        sup_tag_y = 0.3944
        # postion of support class icon.
        sup_ico_x = 0.0729+0.0527*supNo
        sup_ico_y = 0.1796
        
        self.click_act(sup_ico_x, sup_ico_y, 0.5)
        self.click_act(sup_tag_x, sup_tag_y, 1.3)
        
        # game start
        # postion of `mission start` tag
        start_x = 0.9281
        start_y = 0.9398
        self.click_act(start_x, start_y, 1)
    
        
    def use_skill(self, skills):
        # position of skills:
        echo_info('START', 'Using skills:')
        ski_x = [0.0542, 0.1276, 0.2010, 0.3021, 0.3745, 0.4469, 0.5521, 0.6234, 0.6958]
        ski_y = 0.8009
        # snap = 0.0734
        for i in skills:
            # SKILL_SLEEP_TIME = 0.8
            self.click_act(ski_x[i], ski_y, 2.5)
              
        
    def attack(self):
        echo_info('INFO', 'Start attacking...')
        # attack icon position:
        atk_ico_x = 0.8823
        atk_ico_y = 0.8444
        self.click_act(atk_ico_x, atk_ico_y, 1)
        
        # use atk card:
        # use ultimate atk card:
        if USE_ULTIMATE:
            # ultimate card position:
            time.sleep(0.5)
            ult_x = [0.3171, 0.5005, 0.6839]
            ult_y = 0.2833
            for x in ult_x:
                self.click_act(x, ult_y, 0.1)
        
        # use normal atk card:
        # normal atk card position:
        atk_card_x = [0.1003+0.2007*x for x in range(5)]
        atk_card_y = 0.7019
        for i in range(3):
            self.click_act(atk_card_x[i], atk_card_y, 0.1)
            
            
    def surveil(self, x1, y1, x2, y2, name=None, save_img=False):
        last_img = self.pic_shot_float(x1, y1, x2, y2)
        last_diff = 0
        for i in range(100):
            beg = time.time()      
            time.sleep(1)
            if save_img:
                img = self.pic_shot_float(x1, y1, x2, y2)
                img.save('./data/{}_{}.jpg'.format(name, i))
            
            now_diff = compare_img(img, last_img)
            print('[DEBUG] now_diff={}, last_diff={}'.format(now_diff, last_diff))
            if now_diff > 5 and now_diff > last_diff:
                echo_info('SURVEIL', 'Detected status change.')
                return 1
            last_diff = now_diff
            last_img = img
            end = time.time()
            print('[SURVEIL {}] Surveiling in area {}, Time use: {}'.format(
                time.strftime('%H:%M:%S'), name, end-beg))
        return -1
    
    
    def pic_shot_float(self, float_x1, float_y1, float_x2, float_y2, name=None):
        # error: 关闭屏幕缩放！关闭屏幕缩放！
        x1, y1 = self._set(float_x1, float_y1)
        #self.click_act(x1, y1, 1)
        x2, y2 = self._set(float_x2, float_y2)
        # self.click_act(x2, y2, 1)
        return pic_shot(x1, y1, x2, y2, name)
        
    
    def wait_loading(self, save_img=False, algo=0, sleep=None, mode=0):
        '''
        sample in the attack icon per 1s, if loading process is over, break the loop.
        '''
        real_atk = self.atk_img
        real_loading = self.loading_img
        
        # sample1 area of attack icon:
        smp1_x1 = 0.8708
        smp1_y1 = 0.7556
        smp1_x2 = 0.8979
        smp1_y2 = 0.8009
        
        for i in range(100):
            now_atk_img = self.pic_shot_float(smp1_x1, smp1_y1, smp1_x2, smp1_y2)
            if save_img:
                now_atk_img.save('./data/now_loading_{}.jpg'.format(i))
            echo_info('TRY', 'Waiting for loading...')
            diff1 = compare_img_new(now_atk_img, real_atk, algo)
            diff2 = compare_img_new(now_atk_img, real_loading, algo)
            
            if DEBUG:
                print('[DEBUG] different between now and ATK is:', diff1)
                print('[DEBUG] different between now and LOADING is:', diff2)
            
            if mode == -1:
                condition = (diff2 == 0 and diff1 > 0)
            else:
                condition = (diff1 < diff2 and diff2!=0)
                
            if condition:
                time.sleep(0.8)
                echo_info('INFO', 'Detected status change.')
                return diff1
                if sleep:
                    # wait for background anime finishing:
                    time.sleep(sleep)
            time.sleep(1)
        return -1
    
    
    def cal_diff(self, x1, y1, x2, y2, target, save_img=False, hash=True):
        '''
        sample in the position of attack icon to find that if the game is in the loading page.
        return a BOOL type data.
        - x1, y1, x2, y2: the position of the origin area.
        - save_img: if you want to save images.
        - hash: use hash algorithm or compare image simply. 
        '''        
        real_loading = target
        now_atk_img = self.pic_shot_float(x1, y1, x2, y2)
        if save_img:
            now_atk_img.save('./data/sample.jpg')
        if hash:
            return compare_img_new(now_atk_img, real_loading, 0)
        else:
            return 0 if now_atk_img==self.atk_img else -1
        
    
    def cal_atk_diff(self, targrt, hash=True):
        # sample1 area of attack icon:
        smp1_x1 = 0.8708
        smp1_y1 = 0.7556
        smp1_x2 = 0.8979
        smp1_y2 = 0.8009
        
        return self.cal_diff(smp1_x1, smp1_y1, smp1_x2, smp1_y2, self.atk_img, hash)
    
        
    def one_turn(self):
        '''
        to run just one-turn battle.
        '''
        NUM = 2
        # Trun number position: (1298, 123), (1335, 164)
        # turn_num_x1 = 0.6760
        # turn_num_y1 = 0.1139
        # turn_num_x2 = 0.6953
        # turn_num_y2 = 0.1519
        
        # uodate saved atk icon:
        self.diff_atk = self.cal_atk_diff(targrt=self.atk_img)
        print('save new atk diff:', self.diff_atk)
        
        self.attack()
        time.sleep(3)
        
        diffs = []
        # compare atk icon to the last saved icon.
        while 1:
            diff = self.cal_atk_diff(targrt=self.atk_img)
            echo_info('STATUS', 'sampling in the area1, diff=:{}'.format(diff))
            if diff == self.diff_atk:
                n = 0
                for _ in range(NUM):
                    tmp = self.cal_atk_diff(targrt=self.atk_img)
                    time.sleep(0.1)
                    echo_info('INFO', 'last test: now diff=:{}'.format(tmp))
                    if tmp == self.diff_atk:
                        n += 1
                if n==NUM:
                    return 0
            else:
                diffs.append(diff)
                if diffs[-BATTLE_END_TIMEOUT:] == [diff]*BATTLE_END_TIMEOUT and diff != LOADING_ATK_DIFF:
                    echo_info('STATUS', 'Detected status changing, battle finished.')
                    print('------------------------[BATTLE FINISH]--------------------------------')
                    return 1
                
            # click to skip something.
            self.click_act(0.4240, 0.1296, 0)
            time.sleep(SURVEIL_TIME_OUT)
        # res = self.surveil(turn_num_x1, turn_num_y1, turn_num_x2, turn_num_y2, name='[TURN_NUMBER]', save_img=DEBUG)

        
    def one_turn_new(self):   
        # sample1 area of attack icon:(1672, 816), (1724, 865)
        smp1_x1 = 0.8708
        smp1_y1 = 0.7556
        smp1_x2 = 0.8979
        smp1_y2 = 0.8009
        # uodate saved atk icon:
        self.atk_img = self.pic_shot_float(smp1_x1, smp1_y1, smp1_x2, smp1_y2)
        print('save new atk img')
        
        self.attack()
        time.sleep(3)
        
        # compare atk icon to the last saved icon.
        while 1:
            diff = self.cal_atk_diff(targrt=self.atk_img, hash=False)
            print('now diff is :', diff)
            if diff == 0:
                return diff
            time.sleep(SURVEIL_TIME_OUT)
            
    
    def one_battle(self, go_on=False):
        '''
        main part of running the program. 
        '''
        if not go_on:
            self.enter_battle(SUPPORT)
            # wait for going into loading page:
            time.sleep(3)
            self.diff_atk = self.wait_loading(save_img=DEBUG, sleep=5)
            self.use_skill(USED_SKILL)
        
        for i in range(30):
            print('---------------------Turn {}---------------------'.format(i))
            over = self.one_turn()
            if over:
            
                # self.click_act(0.5, 0.5, 2)
                # self.click_act(0.5, 0.5, 1)
                
                
                # 'next' icon.
                end_x = 0.8667
                end_y = 0.9417
                self.click_act(end_x, end_y, 1)
                
                break

    def run(self):
        for _ in range(EPOCH):
            self.one_battle()
            time.sleep(8)
                    
                  

        

In [4]:
fgo = Fgo(full_screen=False, sleep=False)
fgo.one_battle(go_on=False)

[INFO 11:46:50] Move cursor to the top-left cornor of Fgo screen, and press ENTER(enter q to exit):
[INFO 11:46:51] Get cursor at (29, 188)
[INFO 11:46:51] Move cursor to the down-right cornor of Fgo screen, and press ENTER(enter q to exit):
[INFO 11:46:53] Get cursor at (1082, 778)
[INFO 11:46:53] Continue?[y/n](y to continue, n to reset cursor):

---------------------[init over]---------------------------
---------------------Turn 0---------------------
save new atk diff: 7
[INFO 11:46:55] Start attacking...
now diff is : 33
[33]
now diff is : 33
[33, 33]
now diff is : 33
[33, 33, 33]
now diff is : 33
[33, 33, 33, 33]
now diff is : 33
[33, 33, 33, 33, 33]
now diff is : 33
[33, 33, 33, 33, 33, 33]
now diff is : 33
[33, 33, 33, 33, 33, 33, 33]
now diff is : 21
[33, 33, 33, 33, 33, 33, 33, 21]
now diff is : 29
[33, 33, 33, 33, 33, 33, 33, 21, 29]
now diff is : 36
[33, 33, 33, 33, 33, 33, 33, 21, 29, 36]
now diff is : 36
[33, 33, 33, 33, 33, 33, 33, 21, 29, 36, 36]
now diff is : 20
[33, 

now diff is : 8
[29, 21, 22, 26, 21, 27, 35, 31, 20, 30, 19, 25, 41, 17, 14, 29, 29, 31, 20, 8]
now diff is : 22
[29, 21, 22, 26, 21, 27, 35, 31, 20, 30, 19, 25, 41, 17, 14, 29, 29, 31, 20, 8, 22]
now diff is : 7
last test: now diff: 7
last test: now diff: 7
---------------------Turn 4---------------------
save new atk diff: 7
[INFO 11:48:14] Start attacking...
now diff is : 34
[34]
now diff is : 34
[34, 34]
now diff is : 36
[34, 34, 36]
now diff is : 34
[34, 34, 36, 34]
now diff is : 35
[34, 34, 36, 34, 35]
now diff is : 57
[34, 34, 36, 34, 35, 57]
now diff is : 17
[34, 34, 36, 34, 35, 57, 17]
now diff is : 9
[34, 34, 36, 34, 35, 57, 17, 9]
now diff is : 17
[34, 34, 36, 34, 35, 57, 17, 9, 17]
now diff is : 29
[34, 34, 36, 34, 35, 57, 17, 9, 17, 29]
now diff is : 35
[34, 34, 36, 34, 35, 57, 17, 9, 17, 29, 35]
now diff is : 36
[34, 34, 36, 34, 35, 57, 17, 9, 17, 29, 35, 36]
now diff is : 38
[34, 34, 36, 34, 35, 57, 17, 9, 17, 29, 35, 36, 38]
now diff is : 31
[34, 34, 36, 34, 35, 57, 17,

KeyboardInterrupt: 

In [5]:
x, y = (814, 140)
x/1920, y/1080

(0.4239583333333333, 0.12962962962962962)

In [8]:
smp1_x1 = 0.8708
smp1_y1 = 0.7556
smp1_x2 = 0.8979
smp1_y2 = 0.8009

smp1_x2*1920, smp1_y2*1080

(1723.968, 864.972)